In [2]:
import os
import argparse

from time import time
from datetime import datetime

import pandas as pd
import json
from libcomcat.search import count, get_event_by_id, search

#lon min, lon max, lat min, lat max
#sw lon, ne lon, sw lat, ne lat

In [3]:
from libcomcat.dataframes import (get_detail_data_frame, get_dyfi_data_frame,
                                  get_history_data_frame, get_magnitude_data_frame,
                                  get_pager_data_frame, get_phase_dataframe,
                                  get_summary_data_frame)

In [4]:
with open('countries_bbox.json', encoding = 'utf-8') as j:
        co_bb = json.load(j)

In [5]:
co_list = []
for co_obj in co_bb:
    co_list.append(co_obj)

In [6]:
co_list[0]

'ABW'

In [12]:
country = 'USA'
dt_start = '1990-01-01'
dt_end = '1990-04-01'
mag_start = 4
mag_end = 5
headers = ['event_id','datetime', 'location', 'latitude', 'longitude', 'depth', 'magnitude', 'significance', 'alert', 'url', 'eventtype' ,'country']

In [8]:
get_all_countries = [co_obj for co_obj in co_bb]

In [ ]:
get_all_countries

In [ ]:
eq_events = []

In [13]:
temp_list = ['USA', 'ABW']

In [14]:
eq_events = []
for country in temp_list:
    mag_events = search(starttime=datetime(int(dt_start[0:4]), int(dt_start[5:7]), int(dt_start[8:11])), 
                        endtime=datetime(int(dt_end[0:4]), int(dt_end[5:7]), int(dt_end[8:11])),
                        minlatitude=co_bb[country]['sw']['lat'], 
                        maxlatitude=co_bb[country]['ne']['lat'], 
                        minlongitude=co_bb[country]['sw']['lon'], 
                        maxlongitude=co_bb[country]['ne']['lon'],
                        minmagnitude=mag_start, maxmagnitude=mag_end)
    eq_events.extend(mag_events)

In [15]:
eq_events

[ci1049754 1990-01-11 01:22:10.300000 (35.225,-118.215) 2.6 km M4.0,
 nc10090563 1990-01-12 09:10:22.380000 (36.423,-120.786) 12.9 km M4.3,
 usp000446d 1990-01-13 05:47:33.520000 (30.222,-114.440) 10.0 km M4.6,
 usp00044a8 1990-01-15 05:29:03.450000 (37.988,-118.210) 5.0 km M4.4,
 nc10090570 1990-01-18 11:45:25.730000 (41.171,-123.765) 34.8 km M4.9,
 usp00044rd 1990-01-26 02:20:17.440000 (26.083,-110.123) 10.0 km M4.8,
 usp00044se 1990-01-27 06:28:52.070000 (30.121,-113.730) 10.0 km M4.2,
 usp00044tn 1990-01-28 04:59:59.190000 (43.313,-102.504) 5.0 km M4.0,
 usp00044ve 1990-01-29 13:16:10.680000 (34.463,-106.879) 12.0 km M4.5,
 usp00044x0 1990-01-31 01:08:19.290000 (34.445,-106.860) 10.0 km M4.0,
 nc10090715 1990-02-06 18:14:06.620000 (34.925,-121.132) 7.8 km M4.2,
 nc10090717 1990-02-07 14:12:14.790000 (36.956,-121.682) 6.7 km M4.1,
 ci1052624 1990-02-18 15:52:59.940000 (33.509,-116.452) 7.9 km M4.3,
 nc1172653 1990-02-26 15:16:41.480000 (37.463,-118.749) 11.9 km M4.0,
 ci1053648 1990

In [ ]:
event_list = []
for events in eq_events:
    events = events.toDict()
    event_list.append([events['id'], events['time'], events['location'], events['latitude'], events['longitude'],
                 events['depth'], events['magnitude'], events['significance'], events['alert'],
        events['url'], events['eventtype'], country])

df=pd.DataFrame(event_list,columns=headers)

In [ ]:
event_list

In [ ]:
df.dtypes

In [ ]:
dt_start = '1990-01-01'
dt_end = '1990-04-01'
mag_start = 4
mag_end = 8

In [ ]:
get_all_countries

In [ ]:
def pull_from_api(dt_start, dt_end, mag_start, mag_end, country, co_bb):
    eq_events = []
    mag_events = search(starttime=datetime(int(dt_start[0:4]), int(dt_start[5:7]), int(dt_start[8:11])), 
                        endtime=datetime(int(dt_end[0:4]), int(dt_end[5:7]), int(dt_end[8:11])),
                        minlatitude=co_bb[country]['sw']['lat'], 
                        maxlatitude=co_bb[country]['ne']['lat'], 
                        minlongitude=co_bb[country]['sw']['lon'], 
                        maxlongitude=co_bb[country]['ne']['lon'],
                        minmagnitude=mag_start, maxmagnitude=mag_end)
    eq_events.extend(mag_events)
    return eq_events

In [ ]:
eq_events

In [ ]:
event_list = []
eq_events = []
countries = ['USA', 'ABW','TTO', 'TUN']
with open('countries_bbox.json', encoding = 'utf-8') as j:
    co_bb = json.load(j)
for country in get_all_countries:
    try:
        eq_events = pull_from_api(dt_start, dt_end, mag_start, mag_end, country, co_bb)
        event_list.extend(eq_events)
    except Exception:
        pass


In [ ]:
list(co_bb.keys())

In [ ]:
co_bb

In [ ]:
summary_df = get_detail_data_frame(summary_events)

In [ ]:
summary_df

In [ ]:
len(df)

In [ ]:
event_dtypes = {
    'event_id' : str,
    'location' : str,
    'latitude' : float,
    'longitude' : float,
    'depth' : float,
    'magnitude' : float,
    'significance' : pd.Int64Dtype(),
    'alert' : str, 
    'url' : str,
    'eventtype' : str,
    'country' : str,
}

In [ ]:
df = df.astype(event_dtypes)

In [ ]:
df['date'] = pd.to_datetime(df['time'].dt.date)
df['timestamp'] = df['time'].dt.time

In [ ]:
df.dtypes

In [ ]:
event_dtypes = {
    'event_id' : str,
    'location' : str,
    'latitude' : float,
    'longitude' : float,
    'depth' : float,
    'magnitude' : float,
    'significance' : pd.Int64Dtype(),
    'alert' : str, 
    'url' : str,
    'eventtype' : str,
    'country' : str,
}

In [8]:
mag_events = search(starttime=datetime(int(dt_start[0:4]), int(dt_start[5:7]), int(dt_start[8:11])), 
                        endtime=datetime(int(dt_end[0:4]), int(dt_end[5:7]), int(dt_end[8:11])),
                        minlatitude=co_bb[country]['sw']['lat'], 
                        maxlatitude=co_bb[country]['ne']['lat'], 
                        minlongitude=co_bb[country]['sw']['lon'], 
                        maxlongitude=co_bb[country]['ne']['lon'],
                        minmagnitude=mag_start, maxmagnitude=mag_end)

In [9]:
summary_df = get_summary_data_frame(mag_events)

In [11]:
summary_df

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
0,ci1049754,1990-01-11 01:22:10.300,"20km N of Mojave, CA",35.225000,-118.215000,2.591,4.01,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,247
1,nc10090563,1990-01-12 09:10:22.380,"36 km WSW of Three Rocks, California",36.423167,-120.785667,12.927,4.30,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,284
2,usp000446d,1990-01-13 05:47:33.520,"96 km SSE of San Felipe, Mexico",30.222000,-114.440000,10.000,4.60,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,326
3,usp00044a8,1990-01-15 05:29:03.450,"30 km NE of Benton, California",37.988000,-118.210000,5.000,4.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,298
4,nc10090570,1990-01-18 11:45:25.730,"27 km ESE of Orick, California",41.171167,-123.764667,34.760,4.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,369
5,usp00044rd,1990-01-26 02:20:17.440,"80 km W of El Aguajito, Mexico",26.083000,-110.123000,10.000,4.80,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,354
6,usp00044se,1990-01-27 06:28:52.070,"102 km WNW of La Libertad, Mexico",30.121000,-113.730000,10.000,4.20,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,271
7,usp00044tn,1990-01-28 04:59:59.190,"8 km N of Manderson-White Horse Creek, South D...",43.313000,-102.504000,5.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,246
8,usp00044ve,1990-01-29 13:16:10.680,"6 km W of Abeytas, New Mexico",34.463000,-106.879000,12.000,4.50,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,312
9,usp00044x0,1990-01-31 01:08:19.290,"5 km WSW of Abeytas, New Mexico",34.445000,-106.860000,10.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,246


In [12]:
lst = ['USA']*20

In [14]:
summary_df['country'] = lst

In [15]:
summary_df

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance,country
0,ci1049754,1990-01-11 01:22:10.300,"20km N of Mojave, CA",35.225000,-118.215000,2.591,4.01,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,247,USA
1,nc10090563,1990-01-12 09:10:22.380,"36 km WSW of Three Rocks, California",36.423167,-120.785667,12.927,4.30,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,284,USA
2,usp000446d,1990-01-13 05:47:33.520,"96 km SSE of San Felipe, Mexico",30.222000,-114.440000,10.000,4.60,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,326,USA
3,usp00044a8,1990-01-15 05:29:03.450,"30 km NE of Benton, California",37.988000,-118.210000,5.000,4.40,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,298,USA
4,nc10090570,1990-01-18 11:45:25.730,"27 km ESE of Orick, California",41.171167,-123.764667,34.760,4.90,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,369,USA
5,usp00044rd,1990-01-26 02:20:17.440,"80 km W of El Aguajito, Mexico",26.083000,-110.123000,10.000,4.80,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,354,USA
6,usp00044se,1990-01-27 06:28:52.070,"102 km WNW of La Libertad, Mexico",30.121000,-113.730000,10.000,4.20,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,271,USA
7,usp00044tn,1990-01-28 04:59:59.190,"8 km N of Manderson-White Horse Creek, South D...",43.313000,-102.504000,5.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,246,USA
8,usp00044ve,1990-01-29 13:16:10.680,"6 km W of Abeytas, New Mexico",34.463000,-106.879000,12.000,4.50,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,312,USA
9,usp00044x0,1990-01-31 01:08:19.290,"5 km WSW of Abeytas, New Mexico",34.445000,-106.860000,10.000,4.00,None,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,246,USA


In [9]:
def search_data(dt_start, dt_end, mag_start, mag_end, country, co_bb):
    mag_events = search(starttime=datetime(int(dt_start[0:4]), int(dt_start[5:7]), int(dt_start[8:11])), 
                        endtime=datetime(int(dt_end[0:4]), int(dt_end[5:7]), int(dt_end[8:11])),
                        minlatitude=co_bb[country]['sw']['lat'], 
                        maxlatitude=co_bb[country]['ne']['lat'], 
                        minlongitude=co_bb[country]['sw']['lon'], 
                        maxlongitude=co_bb[country]['ne']['lon'],
                        minmagnitude=mag_start, maxmagnitude=mag_end)
    return mag_events, [country]*len(mag_events)


In [10]:
bob = search_data(dt_start, dt_end, mag_start, mag_end, 'USA', co_bb)

In [11]:
get_detail_data_frame(bob[0])

,id,time,latitude,longitude,depth,magnitude,location,magtype,url,eventtype,alert,significance,nc_np1_strike,nc_np1_dip,nc_np1_rake,nc_np2_strike,nc_np2_dip,nc_np2_rake
0,ci1049754,1990-01-11 01:22:10.300,35.225000,-118.215000,2.591,4.01,"20km N of Mojave, CA",ml,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,247,NaN,NaN,NaN,NaN,NaN,NaN
1,nc10090563,1990-01-12 09:10:22.380,36.423167,-120.785667,12.927,4.30,"36 km WSW of Three Rocks, California",ml,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,284,25.0,70.0,-80.0,178.0,22.0,-116.0
2,usp000446d,1990-01-13 05:47:33.520,30.222000,-114.440000,10.000,4.60,"96 km SSE of San Felipe, Mexico",mb,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,326,NaN,NaN,NaN,NaN,NaN,NaN
3,usp00044a8,1990-01-15 05:29:03.450,37.988000,-118.210000,5.000,4.40,"30 km NE of Benton, California",mb,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,298,NaN,NaN,NaN,NaN,NaN,NaN
4,nc10090570,1990-01-18 11:45:25.730,41.171167,-123.764667,34.760,4.90,"27 km ESE of Orick, California",ml,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,369,20.0,40.0,-110.0,225.0,53.0,-74.0
5,usp00044rd,1990-01-26 02:20:17.440,26.083000,-110.123000,10.000,4.80,"80 km W of El Aguajito, Mexico",mb,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,354,NaN,NaN,NaN,NaN,NaN,NaN
6,usp00044se,1990-01-27 06:28:52.070,30.121000,-113.730000,10.000,4.20,"102 km WNW of La Libertad, Mexico",mb,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,271,NaN,NaN,NaN,NaN,NaN,NaN
7,usp00044tn,1990-01-28 04:59:59.190,43.313000,-102.504000,5.000,4.00,"8 km N of Manderson-White Horse Creek, South D...",mblg,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,246,NaN,NaN,NaN,NaN,NaN,NaN
8,usp00044ve,1990-01-29 13:16:10.680,34.463000,-106.879000,12.000,4.50,"6 km W of Abeytas, New Mexico",mb,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,312,NaN,NaN,NaN,NaN,NaN,NaN
9,usp00044x0,1990-01-31 01:08:19.290,34.445000,-106.860000,10.000,4.00,"5 km WSW of Abeytas, New Mexico",mblg,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,None,246,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
bob[1]

['USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA']